In [1]:
"""
run 11_unblind_each_GRB.py

~12 cpu min per job
"""

'\nrun 11_unblind_each_GRB.py\n\n~12 cpu min per job\n'

In [2]:
import sys
import os
import glob
import numpy as np
import pandas as pd
sys.path.append('../../../')
from greco_grb.scripts.utils import *

In [3]:
from greco_grb.scripts import SETTING
paths = SETTING.PATH(osg=False)
print(paths)
LOCATION = paths.LOCATION
USER = paths.USER
ICDATA_DIR = paths.ICDATA_DIR
DATA_DIR = paths.DATA_DIR
ANA_DIR = paths.ANA_DIR


#### Working at GT ####

LOCATION: 	 GT
USER: 	 cchen641
ICDATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/analyses/online_GRECO/candidate-002-p10
DATA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data
ANA_DIR: 	 /storage/home/hhive1/cchen641/data/icecube/data/greco_grb/data/csky_output



In [4]:
df = pd.read_pickle(DATA_DIR+"/grbwebgbm/grbweb_gbm_noHealpix_2268.pkl")

In [5]:
### create arguments arrays ---> 11.txt
# 0: grb_idx_start
# 1: grb_idx_end
# 2: cpu

# Check 1 out of 2

In [6]:
####### Check before running ########
script_path    = '/scratch/cjchen/submit/11.sub'
## [::-1] reverse order
step = 6
grb_idx_starts = np.r_[0:2268:step]
grb_idx_ends = np.r_[step:2268+step:step]
ncpu = 4
#######################################

In [14]:
contents = ""
for grb_idx_start,grb_idx_end in zip(grb_idx_starts,grb_idx_ends):
    include = False
    for grb_idx in range(grb_idx_start, grb_idx_end):
        grb_name = df.grb_name[grb_idx]
        if not os.path.exists(ANA_DIR+f"/unblind/ts_ns_p/{grb_name}_ts_ns_p.npy"):
            print(f"File does not exists: {grb_name}_ts_ns_p.npy", end="\n")
            include = True
            continue
    if include:
        contents += "{}\t".format(grb_idx_start)
        contents += "{}\t".format(grb_idx_end)
        contents += "{}\n".format(ncpu)

with open("11.txt", 'w') as writer:
    writer.write(contents)
    

File does not exists: GRB190916A_ts_ns_p.npy
File does not exists: GRB190915A_ts_ns_p.npy
File does not exists: GRB190914A_ts_ns_p.npy
File does not exists: GRB190913A_ts_ns_p.npy
File does not exists: GRB190910A_ts_ns_p.npy
File does not exists: GRB190906B_ts_ns_p.npy
File does not exists: GRB190905A_ts_ns_p.npy
File does not exists: GRB190904A_ts_ns_p.npy
File does not exists: GRB160228A_ts_ns_p.npy
File does not exists: GRB160227B_ts_ns_p.npy
File does not exists: GRB160227A_ts_ns_p.npy
File does not exists: GRB160226A_ts_ns_p.npy
File does not exists: GRB160225C_ts_ns_p.npy
File does not exists: GRB160225B_ts_ns_p.npy
File does not exists: GRB160225A_ts_ns_p.npy
File does not exists: GRB160224A_ts_ns_p.npy
File does not exists: GRB160223C_ts_ns_p.npy
File does not exists: GRB160223B_ts_ns_p.npy
File does not exists: GRB160223A_ts_ns_p.npy
File does not exists: GRB160222A_ts_ns_p.npy
File does not exists: GRB160221A_ts_ns_p.npy
File does not exists: GRB160220B_ts_ns_p.npy
File does 

In [15]:
print(contents)

168	174	4
174	180	4
1182	1188	4
1188	1194	4
1194	1200	4



In [9]:
print("Total number of jobs: {}\n".format(contents.count('\n')))

Total number of jobs: 5



In [10]:
### create pbs file ---> 11.pbs

# Check 2 out of 2

In [11]:
nameofjob  = "11"
ntotal     = 5           # copy from the above 2nd cell "Total number of jobs: xxx"
inputtxt   = "11.txt"      # copy from the above 3rd cell
outdir     = ensure_dir("/storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out")
walltimehr = 1  # in Hour


In [12]:
contents = ""
contents += "#PBS -N {}\n".format(nameofjob)
############ update ncpu below ############
contents += "#PBS -l nodes=1:ppn={}\n".format(4)           # number of nodes and cores per node required
contents += "#PBS -l pmem=4gb\n"                             # memory per core
contents += "#PBS -l walltime={}:00:00\n".format(walltimehr)  # dd:hh:mm:ss
contents += "#PBS -q hive\n"
contents += "#PBS -j oe\n"
contents += "#PBS -o {}\n".format(outdir+"/{}.out".format(nameofjob))
contents += "#PBS -t 1-{}%400".format(ntotal)
contents += "\n"
contents += "cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts\n"
contents += "icpy3\n" 
# Get Paramters from input.txt file using $PBS_ARRAYID as the line number 
contents += 'params=`sed -n "${{PBS_ARRAYID}} p" {}`\n'.format("hive_sub/"+inputtxt)    # outer single quote to preserve inner double quotes
contents += "paramsArray=($params)\n"
############ update args below ############
contents += "arg0=${paramsArray[0]}\n"
contents += "arg1=${paramsArray[1]}\n"
contents += "arg2=${paramsArray[2]}\n"
############ update script below ############
contents += "python 11_unblind_each_GRB.py "
############ update arg calls below ############
contents += "--grb_idx_start $arg0 --grb_idx_end $arg1 --ncpu $arg2 "
print(contents)
############ update pbs name below ############
with open("11.pbs", 'w') as writer:
    writer.write(contents)

#PBS -N 11
#PBS -l nodes=1:ppn=4
#PBS -l pmem=4gb
#PBS -l walltime=1:00:00
#PBS -q hive
#PBS -j oe
#PBS -o /storage/home/hhive1/cchen641/scratch/icecube/job_pbs_out/out/11.out
#PBS -t 1-5%400
cd /storage/home/hhive1/cchen641/icecube/greco_grb/scripts
icpy3
params=`sed -n "${PBS_ARRAYID} p" hive_sub/11.txt`
paramsArray=($params)
arg0=${paramsArray[0]}
arg1=${paramsArray[1]}
arg2=${paramsArray[2]}
python 11_unblind_each_GRB.py --grb_idx_start $arg0 --grb_idx_end $arg1 --ncpu $arg2 
